### Dependencies 

In [ ]:
!pip install tensorflow
!pip install scikit.learn
!pip install pandas
!pip install numpy
!pip install seaborn

In [1]:
%matplotlib inline 
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from keras.datasets import mnist
from sklearn.preprocessing import scale
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from tensorflow.keras import layers,losses,models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.applications.imagenet_utils import preprocess_input as zero_mean
from keras.callbacks import EarlyStopping

In [ ]:
device_name = tf.test.gpu_device_name()
print(device_name)

### Data Preprocessing 

In [2]:
(data_train,label_train),(data_test,label_test)=mnist.load_data()
data_train_reshaped= data_train.reshape(data_train.shape[0], -1)
data_test_reshaped= data_test.reshape(data_test.shape[0], -1)

In [ ]:
data_train_reshaped=data_train_reshaped/255
data_test_reshaped=data_test_reshaped/255
data_train_reshaped=scale(data_train_reshaped)
data_test_reshaped=scale(data_test_reshaped)

In [ ]:
data_complete=np.concatenate((data_train_reshaped,data_test_reshaped))
data_complete=data_complete/255.0
data_complete=scale(data_complete)
label_complete=np.concatenate((label_train,label_test))
data_complete=data_complete.tolist()
label_complete=label_complete.tolist()


#### Visualising Data 



In [ ]:
dictdata={'Image': [],'Label':[]}
for i in range(6):
  dictdata['Image'].append(data_train[i][:])
  dictdata['Label'].append(label_train[i])

In [ ]:
displaydf=pd.DataFrame(dictdata)

In [ ]:
f, axarr = plt.subplots(2,3)
for i in range(6):
  axarr[int(i/3),i%3].imshow(displaydf['Image'][i],cmap='gray')

In [ ]:
dictdatatest={'Image': [],'Label':[]}
for i in range(6):
  dictdatatest['Image'].append(data_test[i][:])
  dictdatatest['Label'].append(label_test[i])

displaydftest=pd.DataFrame(dictdatatest)

In [ ]:
#Test Samples

for i in range(6):
  plt.figure()
  plt.title('True Label is %i'%displaydftest['Label'][i])
  plt.imshow(displaydftest['Image'][i],cmap='gray')

In [ ]:
f, axarr = plt.subplots(2,3)
for i in range(6):
  axarr[int(i/3),i%3].imshow(displaydftest['Image'][i],cmap='gray')

In [ ]:
# Complete Dataset Histogram
sns.countplot(label_complete);

### Dimensionality Reduction Need to Visualise

PCA

In [ ]:
# pca=PCA(n_components=459)
pca=PCA(n_components=20)
data_PCA_train_actual=pca.fit_transform(data_train_reshaped)
data_PCA_test_actual=pca.transform(data_test_reshaped)

MDA

In [ ]:
dim_redn_LDA=9
lda=LDA(n_components=dim_redn_LDA)
data_LDA_train_actual=lda.fit_transform(data_train_reshaped,label_train)
data_LDA_test_actual=lda.transform(data_test_reshaped)

### SVM 

Without Dimn Redn (Time Consuming)

In [ ]:
model_svm_linear=SVC(kernel='linear').fit(data_train_reshaped,label_train)
model_svm_linear.score(data_test_reshaped,label_test)

In [ ]:
model_svm_linear.score(data_train_reshaped,label_train )

####Linear Kernel

With PCA

In [ ]:
model_svm_linear=SVC(kernel='linear').fit(data_PCA_train_actual,label_train)
model_svm_linear.score(data_PCA_test_actual,label_test)

In [ ]:
#How well it is fit on the training data

model_svm_linear.score(data_PCA_train_actual,label_train)

In [ ]:
y_pred=model_svm_linear.predict(data_PCA_test_actual)
cm=metrics.confusion_matrix(y_true=label_test, y_pred=y_pred)
plt.figure(figsize = (16,8))
sns.heatmap(cm, annot=True);

With LDA

In [ ]:
model_svm_linear=SVC(kernel='linear').fit(data_LDA_train_actual,label_train)
model_svm_linear.score(data_LDA_test_actual,label_test)

In [ ]:
y_pred=model_svm_linear.predict(data_LDA_test_actual)
cm=metrics.confusion_matrix(y_true=label_test, y_pred=y_pred)
plt.figure(figsize = (16,8))
sns.heatmap(cm, annot=True);

#### RBF Kernel

With PCA

In [ ]:
#Searching for optimal parameters

# param_grid = { "C" : [0.1]
#               , "gamma" : [0.1]}
# rf = SVC()
# gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=2, n_jobs=-1, verbose=1)
# gs = gs.fit(data_PCA_train, label_train)
# bp = gs.best_params_

In [ ]:
# model_svm_linear=SVC(C=bp['C'], kernel='rbf', gamma=bp['gamma']).fit(data_PCA_train,label_train)
model_svm_rbf=SVC( kernel='rbf' ).fit(data_PCA_train_actual,label_train)
model_svm_rbf.score(data_PCA_test_actual,label_test)

In [ ]:
y_pred=model_svm_rbf.predict(data_PCA_test_actual)
cm=metrics.confusion_matrix(y_true=label_test, y_pred=y_pred)
plt.figure(figsize = (16,8))
sns.heatmap(cm, annot=True);

With LDA

In [ ]:
model_svm_rbf=SVC( kernel='rbf' ).fit(data_LDA_train_actual,label_train)
model_svm_rbf.score(data_LDA_test_actual,label_test)

In [ ]:
y_pred=model_svm_rbf.predict(data_LDA_test_actual)
cm=metrics.confusion_matrix(y_true=label_test, y_pred=y_pred)
plt.figure(figsize = (16,8))
sns.heatmap(cm, annot=True);

####Polynomial Kernel

With PCA

In [ ]:
model_svm_poly=SVC(kernel='poly').fit(data_PCA_train_actual,label_train)
model_svm_poly.score(data_PCA_test_actual,label_test)

In [ ]:
y_pred=model_svm_poly.predict(data_PCA_test_actual)
cm=metrics.confusion_matrix(y_true=label_test, y_pred=y_pred)
plt.figure(figsize = (16,8))
sns.heatmap(cm, annot=True);

With LDA

In [ ]:
model_svm_poly=SVC(kernel='poly').fit(data_LDA_train_actual,label_train)
model_svm_poly.score(data_LDA_test_actual,label_test)

In [ ]:
y_pred=model_svm_poly.predict(data_LDA_test_actual)
cm=metrics.confusion_matrix(y_true=label_test, y_pred=y_pred)
plt.figure(figsize = (16,8))
sns.heatmap(cm, annot=True);

### Logistic Regression Need to experiment a bit more

In [ ]:
data_train_LR=scale(data_train_reshaped)
data_test_LR=scale(data_test_reshaped)

With PCA

In [ ]:
clf = LogisticRegression(random_state=0,max_iter=2000).fit(data_PCA_train_actual, label_train)
clf.score(data_PCA_test_actual,label_test)

With MDA

In [ ]:
clf = LogisticRegression(random_state=0,max_iter=2000).fit(data_LDA_train_actual, label_train)
clf.score(data_LDA_test_actual,label_test)

With Cross Validation on PCA

In [ ]:
clfnew=LogisticRegressionCV(max_iter=1000,cv=5).fit(data_PCA_train_actual,label_train)
clfnew.score(data_PCA_test_actual,label_test)

With Cross Validation on MDA

In [ ]:
clfnew=LogisticRegressionCV(max_iter=1000,cv=5).fit(data_LDA_train_actual,label_train)
clfnew.score(data_LDA_test_actual,label_test)

Balanced class weights

In [ ]:
clfbal=LogisticRegression(class_weight='balanced',max_iter=2000).fit(data_PCA_train_actual,label_train)
clfbal.score(data_PCA_test_actual,label_test)

L1 penalty

PCA

In [ ]:
clf = LogisticRegression(random_state=0,max_iter=2000,penalty='l1',solver='liblinear').fit(data_PCA_train_actual, label_train)
clf.score(data_PCA_test_actual,label_test)

MDA

In [ ]:
clf = LogisticRegression(random_state=0,max_iter=2000,penalty='l1',solver='liblinear').fit(data_LDA_train_actual, label_train)
clf.score(data_LDA_test_actual,label_test)

Changing Solvers

In [ ]:
clf = LogisticRegression(random_state=0,max_iter=2000,solver='saga').fit(data_PCA_train_actual, label_train)
clf.score(data_PCA_test_actual,label_test)

In [ ]:
clf = LogisticRegression(random_state=0,max_iter=2000,solver='newton-cg').fit(data_PCA_train_actual, label_train)
clf.score(data_PCA_test_actual,label_test)

In [ ]:
clf = LogisticRegression(random_state=0,max_iter=2000,solver='liblinear').fit(data_PCA_train_actual, label_train)
clf.score(data_PCA_test_actual,label_test)

### CNN Implementation Investigate Dropout See early stop conditions


In [ ]:
paddings=tf.constant([[0,0],[2,2],[2,2]])
data_train_padded=tf.pad(data_train,paddings,"CONSTANT")
data_train_padded.shape
data_test_padded=tf.pad(data_test,paddings,"CONSTANT")
data_test_padded.shape

In [ ]:
data_train_padded=data_train_padded/255
data_test_padded=data_test_padded/255

In [ ]:
data_train_padded_array=np.array(data_train_padded)/255.0
data_test_padded_array=np.array(data_test_padded)/255.0

In [ ]:
data_train_CNN= data_train_padded_array.reshape(data_train_padded_array.shape[0], -1)
data_test_CNN= data_test_padded_array.reshape(data_test_padded_array.shape[0], -1)
data_train_CNN.shape

In [ ]:
data_train_CNN_array=data_train_CNN/255.0
data_test_CNN_array=data_test_CNN/255.0

In [ ]:
[x,y,z]=data_train_padded.shape

In [ ]:
#Adding Channel information (Grayscale)
data_train_CNN = tf.expand_dims(data_train_padded, axis=3, name=None)
data_test_CNN = tf.expand_dims(data_test_padded, axis=3, name=None)

In [ ]:
#Defining LeNet5

model = models.Sequential()
model.add(layers.Conv2D(6, 5, activation='tanh', input_shape=data_train_CNN.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(120, 5, activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

In [ ]:
model.evaluate(data_test_CNN, label_test)

Same Architecture changing activation functions

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(6, 5, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(16, 5, activation='relu'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(120, 5, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

In [ ]:
model.evaluate(data_test_CNN, label_test)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

Changing Kernel Size

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(6, 3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.Conv2D(6, 3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(120, 3, activation='relu'))
model.add(layers.Conv2D(120, 3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()


In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

In [ ]:
model.evaluate(data_test_CNN, label_test)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

MSE is not good for classification. Non-convex

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(6, 3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.Conv2D(6, 3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(120, 3, activation='relu'))
model.add(layers.Conv2D(120, 3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(10, activation='sigmoid'))
model.summary()


In [ ]:
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

In [ ]:
model.evaluate(data_test_CNN, label_test)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

More tuning

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32,3, activation='relu', input_shape=data_train_CNN.shape[1:]))
model.add(layers.MaxPooling2D(2,strides=(2,2)))
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2,strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128,input_shape=(3136,), activation=None))
model.add(layers.Dense(10, input_shape=(128,), activation=None))
model.summary()


In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(data_train_CNN, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN, label_test))

In [ ]:
model.evaluate(data_test_CNN, label_test)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

In [ ]:
data_train_unpadded=(data_train/255)
data_test_unpadded=(data_test/255)

In [ ]:
data_train_CNN_Le1 = tf.expand_dims(data_train_unpadded, axis=3, name=None)
data_test_CNN_Le1 = tf.expand_dims(data_test_unpadded, axis=3, name=None)

In [ ]:
#Lenet 1
model = models.Sequential()
model.add(layers.Conv2D(4, 5, activation='tanh', input_shape=data_train_CNN_Le1.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(12, 5, activation='tanh'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(data_train_CNN_Le1, label_train, batch_size=64, epochs=40, validation_data=(data_test_CNN_Le1, label_test))

In [ ]:
model.evaluate(data_test_CNN_Le1, label_test)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

Dropout in architecture

In [ ]:
# Lenet 5

model = models.Sequential()
model.add(layers.Conv2D(6, 5, activation='tanh', input_shape=data_train_CNN.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(120, 5, activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

### Transfer Learning

#### Data Loading and Preprocessing

In [3]:
# Importing Dataset

TRAIN_DIR = 'Transfer_Learning_Dataset/training/training'
VAL_DIR = 'Transfer_Learning_Dataset/validation/validation'


In [ ]:
img_width = 224 # determined by the pre-trained model
img_height = 224 # determined by the pre-trained model
batch_size = 32 # we could try other values if we were concerned about the training speed
train_dataset_size = 1098
val_dataset_size = 272
number_classes = 10 # the dataset contains 10 different species

In [ ]:
cols = ['Label','Latin Name', 'Common Name','Train Images', 'Validation Images']
label_df = pd.read_csv("Transfer_Learning_Dataset/monkey_labels.txt", names=cols, skiprows=1)
label_df

In [ ]:
labels = label_df['Common Name']
labels

In [4]:
generate_train_data = ImageDataGenerator(rescale=1./255)
train_generator = generate_train_data.flow_from_directory(TRAIN_DIR,target_size=(120,120),batch_size=32,class_mode='categorical') 


generate_test_data = ImageDataGenerator(rescale=1./255)
test_generator = generate_test_data.flow_from_directory(VAL_DIR,target_size=(120,120),batch_size=32,class_mode='categorical') 

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


TASK 1

In [5]:
early_stop = EarlyStopping(monitor='val_loss',patience=4)

In [6]:
model =models.Sequential()
model.add(layers.Conv2D(filters=32,kernel_size=(4,4),input_shape=(120,120,3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=32,kernel_size=(4,4),input_shape=(120,120,3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 117, 117, 32)      1568      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 58, 58, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 55, 55, 32)        16416     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 27, 27, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 23328)             0         
                                                                 
 dense (Dense)               (None, 128)               2

2021-12-02 18:00:08.546537: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.compile(optimizer='adam', loss=losses.categorical_crossentropy, metrics=['accuracy'])
model.fit(train_generator,validation_data=test_generator,epochs=10)

Epoch 1/10
35/35 [==============================] - 35s 978ms/step - loss: 0.3962 - accuracy: 0.9144 - val_loss: 1.6656 - val_accuracy: 0.5735
Epoch 2/10
35/35 [==============================] - 32s 921ms/step - loss: 0.0457 - accuracy: 0.9863 - val_loss: 1.7840 - val_accuracy: 0.6140
Epoch 3/10
35/35 [==============================] - 34s 974ms/step - loss: 0.0107 - accuracy: 0.9991 - val_loss: 1.7888 - val_accuracy: 0.6213
Epoch 4/10
35/35 [==============================] - 36s 1s/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 1.8786 - val_accuracy: 0.6287
Epoch 5/10
35/35 [==============================] - 34s 972ms/step - loss: 0.0123 - accuracy: 0.9964 - val_loss: 1.8712 - val_accuracy: 0.6324
Epoch 6/10
35/35 [==============================] - 36s 1s/step - loss: 0.0540 - accuracy: 0.9909 - val_loss: 1.7944 - val_accuracy: 0.5956
Epoch 7/10
35/35 [==============================] - 36s 1s/step - loss: 0.0301 - accuracy: 0.9936 - val_loss: 1.9570 - val_accuracy: 0.5919
Epoch 8/

In [8]:
model.evaluate(test_generator)

9/9 [==============================] - 6s 582ms/step - loss: 1.6275 - accuracy: 0.6250


[1.6275296211242676, 0.625]

In [ ]:
LABEL = 'n0'
IMAGE = 'n0021.jpg'
img = load_img(TRAIN_DIR+'/'+LABEL+'/'+IMAGE) # image loaded as PIL object
_ = plt.imshow(img)

TASK 2

In [75]:
# If on MACOSX please refer to https://github.com/ageron/handson-ml/issues/46#issuecomment-311323613 to workaround the bug

from keras.applications.inception_v3 import InceptionV3
model=InceptionV3(weights='imagenet',include_top=False,input_shape=(299,299,3))

In [73]:
generate_train_data = ImageDataGenerator(rescale=1./255)
train_generator = generate_train_data.flow_from_directory(TRAIN_DIR,target_size=(299,299),batch_size=32,class_mode='categorical') 


generate_test_data = ImageDataGenerator(rescale=1./255)
test_generator = generate_test_data.flow_from_directory(VAL_DIR,target_size=(299,299),batch_size=32,class_mode='categorical') 

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [76]:
model.summary()
model.trainable=False

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_754 (Conv2D)            (None, 149, 149, 32  864         ['input_25[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_752 (Batch  (None, 149, 149, 32  96         ['conv2d_754[0][0]']             
 Normalization)                 )                                                      

In [61]:
from keras.models import Model

In [77]:
prediction_layer = layers.Dense(10)
global_average_layer = layers.GlobalAveragePooling2D()
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip('horizontal'),
  layers.RandomRotation(0.2),
])
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

In [78]:
inputs = tf.keras.Input(shape=(299, 299, 3))
x = data_augmentation(inputs)
x = rescale(x)
x = model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [79]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 299, 299, 3)]     0         
                                                                 
 sequential_8 (Sequential)   (None, 299, 299, 3)       0         
                                                                 
 rescaling_6 (Rescaling)     (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d_11  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_5 (Dropout)         (None, 2048)              0         
                                                           

In [81]:
base_learning_rate = 0.0001
intial_epochs=10
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [82]:
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=test_generator)
history.evaluate(test_generator)

Epoch 1/10
35/35 [==============================] - 65s 2s/step - loss: 2.3396 - accuracy: 0.1011 - val_loss: 2.3028 - val_accuracy: 0.0956
Epoch 2/10
33/35 [===========================>..] - ETA: 3s - loss: 2.3260 - accuracy: 0.0928